# 유튜브 채널 키워드 추출

이 프로젝트는 유튜브 채널의 텍스트 데이터를 정제한 후, **TF-IDF**(Term Frequency-Inverse Document Frequency) 기법을 적용하여 주요 키워드를 추출하는 과정을 설명합니다. 이 방법은 각 채널에서 빈번히 사용되지만 다른 채널에서는 상대적으로 드물게 나타나는 단어들을 중요 키워드로 선정합니다.

추출된 키워드는 유입 사용자들이 채널의 핵심 주제를 빠르게 파악하는 데 도움을 주며, 콘텐츠 제작자에게는 새로운 콘텐츠 아이디어를 제공하는 데 활용됩니다.

## 주요 단계:
1. **데이터 정제**: 채널 설명 및 제목 등 텍스트 데이터를 수집하고 불필요한 요소 제거.
2. **TF-IDF 계산**: 각 채널에서 사용된 단어들의 빈도를 기반으로 TF-IDF 값을 계산하여 중요 키워드 선정.
3. **키워드 추출**: TF-IDF 값이 높은 단어들을 주요 키워드로 추출.
4. **결과 활용**: 추출된 키워드를 바탕으로 채널의 핵심 주제를 정의하고, 콘텐츠 아이디어를 도출.

이 과정은 채널의 특성을 명확히 이해하는 데 기여하며, 사용자와의 상호작용을 증대시키기 위한 전략 수립에도 도움이 됩니다.


In [2]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict

In [23]:
# 채널 데이터와 비디오 데이터 불러오기
channel_data = pd.read_csv("../data/channel_sample.csv", encoding="utf-8-sig", index_col=0)
video_data = pd.read_csv("../data/video_sample_add_pre.csv", encoding="utf-8-sig", index_col=0)

In [24]:
# channel_id가 5개 이상인 항목 즉, 영상을 5개 이상 업로드한 채널만 추출
video_data = video_data.groupby('channel_id').filter(lambda x: len(x) >= 5).reset_index(drop=True)
channel_data = channel_data[channel_data.CHANNEL_ID.isin(video_data.channel_id.unique())]

,video_id,channel_id,video_title,video_description,video_tags,video_duration,video_published,video_category,video_info_card,video_with_ads,video_end_screen,video_cluster,crawled_date,year,month,day,pre_text
0,2y3SQ2v5IQk,UC3_oS5M6sufpC03eS2L8fPg,[자막뉴스] 홍수에 떠내려가자 '박수' '알박기 염치' 오죽했으면..,#알박기 #캠핑 #sbs뉴스 #알박기 #텐트 #캠핑,[],358,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:44:27,2024,10,6,[ 자막뉴스 홍수에 떠내려가자 박수 알박기 염치 #알박기 #캠핑 #sbs뉴스 #알...
1,UMnnHdJHx-U,UC3_oS5M6sufpC03eS2L8fPg,"중국 일당에 ""다 들어와""…공포에 떤 유흥가 (자막뉴스)",#범죄도시 #6000 #sbs뉴스 지난 2012년 한국으로 귀화한 중국 연변 출신의...,[],354,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:44:27,2024,10,6,중국 일당에 다 들어와 공포에 떤 유흥가 자막뉴스 #범죄도시 #6000 #sbs뉴...
2,xLz-nqWD8lw,UC3_oS5M6sufpC03eS2L8fPg,"국힘당 문자 하나로 파장! 지지율 폭락 예정! 장철민, 청탁금지법? 권익위원장에게 ...",#노종면 #장철민 #sbs뉴스 #장철민 #인요한 #노종면,[],584,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:44:27,2024,10,6,국힘당 문자 하나로 파장 지지율 장철민 청탁금지법 권익위원장에게 물어보니 노종면의원...
3,UDjGTZi934E,UC3_oS5M6sufpC03eS2L8fPg,한국 건물주 된 중국인 여대생...누리꾼 공분한 까닭,#오클릭 #SBS뉴스 #sbs뉴스 SNS를 통해 오늘(27일) 하루 관심사와 누리꾼...,[],371,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:44:27,2024,10,6,한국 건물주 된 중국인 여대생 누리꾼 공분한 까닭 #오클릭 #SBS뉴스 #sbs뉴...
4,VTiODAvtK8A,UC3_oS5M6sufpC03eS2L8fPg,강유정의원 질의도중 불순한태도 보이는 총무비서관!...오늘 처음으로 한마디하는 박찬...,#박찬대 #강유정 #sbs뉴스 #강유정 #박찬대 #총무비서관,[],616,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:44:27,2024,10,6,강유정의원 질의도중 불순한태도 보이는 총무비서관 처음으로 한마디하는 박찬대 박성준 ...
5,lOOJIrVUXP8,UC3_oS5M6sufpC03eS2L8fPg,이곳에 갇혀 죽을 날만 기다립니다' 북한 내부 주민과의 비밀 인터뷰,#북한 #sbs뉴스 세 명의 북한 주민들은 BBC와의 독점 비밀 인터뷰를 통해 전 ...,[],373,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:44:27,2024,10,6,이곳에 죽을 북한 내부 주민과의 비밀 인터뷰 #북한 #sbs뉴스 [SEP] New...


In [21]:
# 영상이 5개 이상인 채널이 한 개뿐이므로 해당 채널만 분석
channel_data.T

,0
CHANNEL_ID,UC3_oS5M6sufpC03eS2L8fPg
CHANNEL_NAME,간편이슈
CHANNEL_DESCRIPTION,간편이슈 채널입니다.
CHANNEL_TAGS,[]
MAINLY_USED_KEYWORDS,NaN
MAINLY_USED_TAGS,NaN
CHANNEL_COUNTRY,NaN
CHANNEL_LINK,NaN
CHANNEL_SINCE,2017-03-17
CHANNEL_CLUSTER,9


In [94]:
class TextCleaner():
    def __init__(self, josa_path = "../data/kor_josa.txt", 
                 stopwords_path="../data/stopwords.txt"):
        """
        TextCleaner 클래스의 초기화 함수.
        조사(Josa)와 불용어(Stopwords) 목록을 파일에서 읽어들여 리스트로 저장합니다.

        Args:
            josa_path (str): 조사 파일 경로
            stopwords_path (str): 불용어 파일 경로
        """
        
        # 조사 리스트 초기화 및 파일에서 조사 데이터 로드
        self.josa_list = set()
        with open(josa_path, "r", encoding="utf-8-sig") as f:
            for line in f:
                text = line.strip()
                if text:
                    self.josa_list.add(text)
        self.josa_list = list(self.josa_list)  # 리스트 형태로 변환
        
        # 불용어 리스트 초기화 및 파일에서 불용어 데이터 로드
        self.stopwords_list = set()
        with open(stopwords_path, "r", encoding="utf-8-sig") as f:
            for line in f:
                text = line.strip()
                if text:
                    self.stopwords_list.add(text)
        self.stopwords_list = list(self.stopwords_list)  # 리스트 형태로 변환

    def clean_and_filter_text(self, text):
        """
        주어진 텍스트를 정리하고, 불용어 제거 및 조사 제거 작업을 수행하는 함수.

        Args:
            text (str): 처리할 텍스트

        Returns:
            str: 정리 및 필터링된 텍스트, 필터링에 걸리면 빈 문자열을 반환
        """
        
        # 텍스트 양쪽 공백 제거 및 특수문자 처리
        text = text.strip()
        text = self.clean_text(text)
        
        # 불용어 리스트에 포함되어 있으면 빈 문자열 반환
        if text in self.stopwords_list:
            return ""
        
        # 일련번호 패턴에 해당하면 빈 문자열 반환
        if not self.filter_serial_pattern(text):
            return ""

        # 조사 제거 (텍스트가 조사로 끝날 경우 해당 부분을 제거)
        for josa in self.josa_list:
            # 조사 리스트에서 길이가 2 이상이거나 단일 조사(는, 를, 의, 에 등)에 해당하면 처리
            if (text.endswith(josa)) and ((len(josa) >= 2) or (josa in ['는', '를', '의', '에', '와', '들'])):
                return text[:-len(josa)]
        
        # 별도의 처리 필요 없는 경우 원본 텍스트 반환
        return text

    def clean_text(self, word):
        """
        텍스트에서 일부 특수문자를 제거하고 불필요한 공백을 처리하는 함수.

        Args:
            word (str): 처리할 단어

        Returns:
            str: 특수문자 제거 및 공백 정리된 텍스트
        """
        # 일부 특수문자를 공백으로 대체
        for char in ["`", "'", "#", ",", "]", "["]:
            word = word.replace(char, " ")
        
        # 두 개 이상의 연속된 공백을 하나로 줄임, 양쪽 공백 제거
        word = re.sub(r'\s{2,}', ' ', word).strip()
        
        return word
    
    def filter_serial_pattern(self, word):
        """
        특정 문자로 시작하고 숫자로 끝나는 일련번호 같은 단어를 제외하는 함수.

        Args:
            word (str): 검사할 단어

        Returns:
            bool: 조건에 맞으면 False, 그렇지 않으면 True
        """
        # 정규표현식 패턴: 영문자나 한글로 시작하고, 숫자로 끝나는지 확인
        pattern = r'^[가-힣a-zA-Z].*\d$'
        
        # 조건에 맞으면 False 반환 (일련번호로 간주하여 제외), 맞지 않으면 True 반환
        return not bool(re.match(pattern, word))


In [95]:
class KeywordExtractor(TextCleaner):

    def __init__(self, josa_path='./kor_josa.txt', stopwords_path="./stopwords_fin.txt"):
        """
        KeywordExtractor 초기화 메소드.
        josa_path와 stopwords_path를 받아 조사와 불용어 리스트를 초기화합니다.

        Args:
            josa_path (str): 조사가 저장된 파일의 경로.
            stopwords_path (str): 불용어가 저장된 파일의 경로.
        """
        super().__init__(josa_path=josa_path, stopwords_path=stopwords_path)

        # 불용어 리스트 초기화
        self.stopwords = set()
        with open(stopwords_path, "rt", encoding="utf-8-sig") as f:
            while True:
                text = f.readline().strip()
                if text:
                    self.stopwords.add(text)  # 불용어 추가
                if not text:
                    break  # 파일의 끝에 도달하면 중단
        self.stopwords = list(self.stopwords)  # 불용어를 리스트로 변환

    def extract_keywords_from_tfidf(self, docs, threshold_1=0.8, threshold_2=3, ntop=30, keyword_max=None, use_upper=True):
        """
        TF-IDF를 기반으로 문서에서 키워드를 추출하는 메소드.
        
        Args:
            docs (list of str): 처리할 문서 리스트.
            threshold_1 (float): 키워드 등장수 / 문서수의 임계값 (키워드 필터링 조건).
            threshold_2 (int): 키워드 등장수의 임계값 (최소 등장 수).
            ntop (int): 추출할 상위 키워드의 최대 개수.
            keyword_max (int): 키워드의 최대 길이 (None일 경우 제한 없음).
            use_upper (bool): 키워드를 대문자로 변환할지 여부.

        Returns:
            list: 추출된 키워드 리스트.
        """
        # TF-IDF 벡터라이저 초기화
        vector = TfidfVectorizer()

        # TF-IDF 벡터 변환 시도
        try:
            tfidf = vector.fit_transform(docs).toarray()
        except Exception as e:
            return list()  # 오류 발생 시 빈 리스트 반환
        
        # 피처(키워드) 이름 추출
        columns = vector.get_feature_names_out()

        # TF-IDF 결과를 데이터프레임으로 변환
        tfidf = pd.DataFrame(tfidf, columns=columns)

        # 각 키워드가 등장한 문서 수로 변환 (0/1 값으로 변환 후 합산)
        tfidf = tfidf.astype(bool).sum(axis=0)

        # 숫자로만 이루어진 키워드는 제거
        tfidf = tfidf[~tfidf.index.str.isdigit()]

        # TF-IDF 가중치 합계
        sum_score = sum(tfidf)

        # 키워드의 상대적 가중치로 변환 (전체 합으로 나누기)
        tfidf = tfidf / sum_score

        # 키워드를 가중치에 따라 내림차순으로 정렬
        tfidf = tfidf.sort_values(ascending=False)

        # 불용어 제거
        tfidf_result = [word for word in tfidf.index if word not in self.stopwords]
        result = list()

        # 키워드 필터링 및 추출
        for keyword in tfidf_result:
            if len(result) >= ntop:
                break  # 상위 ntop 키워드만 추출
            
            keyword = keyword.strip()  # 키워드 앞뒤 공백 제거

            # 키워드의 길이가 keyword_max보다 크면 패스
            if keyword_max and len(keyword) > keyword_max:
                continue
            # 문서 내에서 해당 키워드의 등장 빈도 계산
            count = sum(text.lower().split().count(keyword.lower()) for text in docs)
            # 필터링 조건 적용: 문서 수 대비 등장 비율 및 최소 등장 수 기준
            if (count / len(docs) <= threshold_1) and (count >= threshold_2):
                # 조사 제거 등의 후처리 진행
                keyword = self.clean_and_filter_text(keyword)
                
                if keyword and len(keyword) > 1:  # 빈 문자열이나 1글자 키워드는 제외
                    if use_upper:  # 대문자로 변환
                        keyword = keyword.upper()
                    result.append(keyword)  # 결과에 추가

        # 중복 키워드 제거한 리스트 반환
        return list(OrderedDict.fromkeys(result))


In [96]:
ke = KeywordExtractor(josa_path="../data/kor_josa.txt",
                 stopwords_path="../data/stopwords_for_keyword.txt")
ke.extract_keywords_from_tfidf(video_data.pre_text.tolist(),
                               threshold_1=1.0, threshold_2=2)


['NEWS', 'POLITICS', '자막뉴스']